# Access non-IPv6 services (i.e. GitHub) from IPv6 FABRIC nodes.



## Import the FABLlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  --------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /Users/pruth/work/fabric_config/fabric_token.json
project_id                           990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username                     pruth_0031379841
bastion_key_filename                 /Users/pruth/work/fabric_config/fabric_bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /Users/pruth/work/fabric_config/slice-public-key
slice_private_key_file               /Users/pruth/work/fabric_config/slice-private-key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  -------------------

## Create a Node

In [2]:
try:
    #Create a slice
    slice = fablib.new_slice(name="MySlice")

    # Add a node (avoid IPv4 sites)
    slice.add_node(name="Node1", avoid=['TACC','MAX'])

    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


---------------  ------------------------------------
Slice Name       MySlice
Slice ID         51360bcc-85bb-4ee8-9bdc-4a45a1f528f9
Slice State      StableOK
Lease End (UTC)  2022-07-18 17:18:20 +0000
---------------  ------------------------------------

Retry: 9, Time: 103 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image            Management IP                          State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  ---------------  -------------------------------------  -------  -------
abb2ced0-e5a4-4007-9fd0-ac0e7a378cd5  Node1   STAR    star-w4.fabric-testbed.net        2      8      10  default_rocky_8  2001:400:a100:3030:f816:3eff:feb2:2dd  Active

Time to stable 103 seconds
Running post_boot_config ... Time to post boot config 104 seconds


### Print the node


In [3]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  -----------------------------------------------------------------------------------------------------------------------------------------------------
ID                 abb2ced0-e5a4-4007-9fd0-ac0e7a378cd5
Name               Node1
Cores              2
RAM                8
Disk               10
Image              default_rocky_8
Image Type         qcow2
Host               star-w4.fabric-testbed.net
Site               STAR
Management IP      2001:400:a100:3030:f816:3eff:feb2:2dd
Reservation State  Active
Error Message
SSH Command        ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3030:f816:3eff:feb2:2dd
-----------------  -----------------------------------------------------------------------------------------------------------------------------------------------------


## Upload and Execute the NAT64 Script

In [4]:
from ipaddress import ip_address, IPv6Address

try:
    node = slice.get_node(name="Node1")     
    
    #If the node is an IPv6 Node then configure NAT64
    if type(ip_address(node.get_management_ip())) is IPv6Address:
        node.upload_file('nat64.sh','nat64.sh')
        
        stdout, stderr = node.execute(f'chmod +x nat64.sh && ./nat64.sh')
        print(stdout)
        print(stderr)
        
    #Access non-IPv6 Services
    stdout, stderr = node.execute(f'sudo yum install -y -q git && git clone https://github.com/fabric-testbed/jupyter-examples.git')
    print(stdout)
    print(stderr)
    
    stdout, stderr = node.execute(f'ls jupyter-examples')
    print(stdout)
    print(stderr)
     
        
except Exception as e:
    print(f"Exception: {e}")




Installed:
  emacs-filesystem-1:26.1-7.el8.noarch                                          
  git-2.31.1-2.el8.x86_64                                                       
  git-core-2.31.1-2.el8.x86_64                                                  
  git-core-doc-2.31.1-2.el8.noarch                                              
  perl-Carp-1.42-396.el8.noarch                                                 
  perl-Data-Dumper-2.167-399.el8.x86_64                                         
  perl-Digest-1.17-395.el8.noarch                                               
  perl-Digest-MD5-2.55-396.el8.x86_64                                           
  perl-Encode-4:2.97-3.el8.x86_64                                               
  perl-Errno-1.28-421.el8.x86_64                                                
  perl-Error-1:0.17025-2.el8.noarch                                             
  perl-Exporter-5.72-396.el8.noarch                                             
  perl-File-Pa

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")